**Tips.csv**

Abrindo base de dados previamente gravada no Drive e analisando as suas variáveis.

Variável de interesse deve ser construída para indicar se uma gorjeta é maior ou menor do que mediana.

total_bill e tip tem que ser dropadas pois há endogeneidade com a variável de interesse.

sex, smoker, day e time precisam ser convertidas.

In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/FIAP/Machine Learning/SearchGrid/tips.csv')
df.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Criando a variável com a relação de tip e total_bill.

Se maior que a mediana, então tip_size = 1 (um boa gorjeta)

Se menor ou igual a mediana, tip_size = 0 (uma gorjeta ruim)

In [2]:
df['tip_size'] = df['tip'] / df['total_bill']
df.head(1)
median = df['tip_size'].median()
df['tip_size'] = df.apply(lambda row: 1 if row['tip_size'] > median else 0, axis=1)


Convertendo o dia da semana em variáveis dummy.

Observar se a distribuição de 'day' bate com as distribuições dos dias da semana individuais.

In [3]:
print(df['day'].value_counts())
df['sun'] = df.apply(lambda row: 1 if row['day'] == 'Sun' else 0, axis=1)
df['sat'] = df.apply(lambda row: 1 if row['day'] == 'Sat' else 0, axis=1)
df['fri'] = df.apply(lambda row: 1 if row['day'] == 'Fri' else 0, axis=1)
df['thu'] = df.apply(lambda row: 1 if row['day'] == 'Thur' else 0, axis=1)
print(df['sun'].value_counts())
print(df['sat'].value_counts())
print(df['fri'].value_counts())
print(df['thu'].value_counts())

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64
sun
0    168
1     76
Name: count, dtype: int64
sat
0    157
1     87
Name: count, dtype: int64
fri
0    225
1     19
Name: count, dtype: int64
thu
0    182
1     62
Name: count, dtype: int64


Convertendo variáveis binárias e conferindo as quantidades.

In [4]:
print(df['time'].value_counts())
print(df['sex'].value_counts())
print(df['smoker'].value_counts())
df['dinner'] = df.apply(lambda row: 1 if row['time'] == 'Dinner' else 0, axis=1)
df['female'] = df.apply(lambda row: 1 if row['sex'] == 'Female' else 0, axis=1)
df['smoke'] = df.apply(lambda row: 1 if row['smoker'] == 'Yes' else 0, axis=1)
print(df['dinner'].value_counts())
print(df['female'].value_counts())
print(df['smoke'].value_counts())


time
Dinner    176
Lunch      68
Name: count, dtype: int64
sex
Male      157
Female     87
Name: count, dtype: int64
smoker
No     151
Yes     93
Name: count, dtype: int64
dinner
1    176
0     68
Name: count, dtype: int64
female
0    157
1     87
Name: count, dtype: int64
smoke
0    151
1     93
Name: count, dtype: int64


Cópia de segurança do meu df original.

Removendo as colunas que não são mais necessárias.

Conferindo se tenho sometne variáveis numéricas no meu dataset.

In [5]:
df1 = df.copy()
del df1['total_bill']
del df1['tip']
del df1['time']
del df1['sex']
del df1['day']
del df1['smoker']
df1.head()


,size,tip_size,sun,sat,fri,thu,dinner,female,smoke
0,2,0,1,0,0,0,1,1,0
1,3,1,1,0,0,0,1,0,0
2,3,1,1,0,0,0,1,0,0
3,2,0,1,0,0,0,1,0,0
4,4,0,1,0,0,0,1,1,0


**Tudo perfeito para iniciar os testes com os classificadores! \o/**


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

x = df1.drop('tip_size', axis='columns')
y = df1['tip_size']

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

params = {
  'random_state':[True],
  'n_estimators': [10,20],
  'max_depth': range(1,20)
}

gs = GridSearchCV(RandomForestClassifier(), param_grid=params, scoring='accuracy')
gs.fit(x, y)
print(gs.score(x, y))
best_estimator = gs.best_estimator_


0.6639344262295082


In [ ]:
gs.best_params_


In [ ]:
results = pd.DataFrame(gs.cv_results_)
results


In [9]:
params = {
  'random_state':[True],
  'criterion': ['log_loss','gini'],
  'max_depth': range(1,15)
}

gs = GridSearchCV(DecisionTreeClassifier(), param_grid=params, scoring='accuracy')
gs.fit(x, y)
print(gs.score(x, y))


0.5491803278688525


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

params = {
  'solver': ['lsqr','svd'],
}

gs = GridSearchCV(LinearDiscriminantAnalysis(), param_grid=params, scoring='accuracy')
gs.fit(x, y)
print(gs.score(x, y))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

y_pred = best_estimator.predict(x)
matrix = confusion_matrix(y, y_pred)
sns.heatmap(matrix/np.sum(matrix), annot=True, fmt='.2%', cmap='Reds')

In [ ]:
# https://www.kaggle.com/datasets/prathamtripathi/drug-classification